<a href="https://colab.research.google.com/github/gamersnsp/CTF-Challanges/blob/main/DragonHunter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mount Gdrive**





In [3]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount Gdrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

Mounted at /content/drive


# **FILE DOWLOADER**

In [6]:
#@markdown <center><h3>-- Url Dowloader --
URL = "https://drive.google.com/file/d/1Gl8U13JS_d2oMlBP2Batl7GOV5Jq2xo9/view?usp=sharing" #@param {type:"string"}
#@markdown <center><h5>Default output path /content/downloads/</h5></center>
OUTPUT_PATH = "" #@param {type:"string"}

import pathlib
import shutil
import hashlib
import requests
from urllib.parse import urlparse
from os import path, mkdir
if not path.exists("/root/.ipython/ocr.py"): 
    from subprocess import run
    from shlex import split

    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py \
                    -O /root/.ipython/ocr.py"
    run(split(shellCmd))
from ocr import runSh

def youtubedlInstall():
  if not path.isfile("/usr/local/bin/youtube-dl"):
    cmdC = "rm -rf /content/sample_data/ && " \
            " mkdir -p -m 666 /root/.YouTube-DL/ &&" \
            " apt-get install atomicparsley &&" \
            " curl -L https://yt-dl.org/downloads/latest/youtube-dl " \
            "-o /usr/local/bin/youtube-dl &&" \
            " chmod a+rx /usr/local/bin/youtube-dl"
    get_ipython().system_raw(cmdC)

def aria2Install():
  runSh('apt install -y aria2')

def istmd(URL): 
  link = urlparse(URL)
    
  #YandexDisk
  if link.netloc == "yadi.sk":
    API_ENDPOINT = 'https://cloud-api.yandex.net/v1/disk/public/resources/' \
                    '?public_key={}&path=/{}&offset={}'
    dry = False
    def md5sum(file_path):
        md5 = hashlib.md5()
        with open(file_path, 'rb') as f:
            for chunk in iter(lambda: f.read(128 * md5.block_size), b''):
                md5.update(chunk)
        return md5.hexdigest()


    def check_and_download_file(target_path, url, size, checksum):
        if path.isfile(target_path):
            if size == path.getsize(target_path):
                if checksum == md5sum(target_path):
                    print('URL {}'.format(url))
                    print('skipping correct {}'.format(target_path))
                    return
        if not dry:
            print('URL {}'.format(url))
            print('downloading {}'.format(target_path))
            runSh(f'aria2c -x 16 -s 16 -k 1M -d {OUTPUT_PATH} {url}', output=True)
            # r = requests.get(url, stream=True)
            # with open(target_path, 'wb') as f:
            #     shutil.copyfileobj(r.raw, f)


    def download_path(target_path, public_key, source_path, offset=0):
        print('getting "{}" at offset {}'.format(source_path, offset))
        current_path = path.join(target_path, source_path)
        pathlib.Path(current_path).mkdir(parents=True, exist_ok=True)
        jsn = requests.get(API_ENDPOINT.format(public_key, source_path, offset)).json()
        def try_as_file(j):
            if 'file' in j:
                file_save_path = path.join(current_path, j['name'])
                check_and_download_file(file_save_path, j['file'], j['size'], j['md5'])
                return True
            return False

        # first try to treat the actual json as a single file description
        if try_as_file(jsn):
            return

        # otherwise treat it as a directory
        emb = jsn['_embedded']
        items = emb['items']
        for i in items:
            # each item can be a file...
            if try_as_file(i):
                continue
            # ... or a directory
            else:
                subdir_path = path.join(source_path, i['name'])
                download_path(target_path, public_key, subdir_path)

        # check if current directory has more items
        last = offset + emb['limit']
        if last < emb['total']:
            download_path(target_path, public_key, source_path, last)
    download_path(OUTPUT_PATH, URL, '')
    return False  
  return URL

if not OUTPUT_PATH:
  OUTPUT_PATH = "/content/downloads/"
  
if not URL == "":
  aria2Install()
  youtubedlInstall()
  try:
    mkdir("downloads")
  except FileExistsError:
    pass
  url = istmd(URL)
  if url != False:
    print('URL {}'.format(URL))
    cmdC = f'youtube-dl -o "{OUTPUT_PATH}/%(title)s" {URL} ' \
            '--external-downloader aria2c ' \
            '--external-downloader-args "-x 16 -s 16 -k 1M"'
    runSh(cmdC, output=True)
else:
  print("Please input url")

URL https://drive.google.com/file/d/1Gl8U13JS_d2oMlBP2Batl7GOV5Jq2xo9/view?usp=sharing
[GoogleDrive] 1Gl8U13JS_d2oMlBP2Batl7GOV5Jq2xo9: Downloading webpage
ERROR: This video doesn't exist.


**MEGA File Dowload**

In [ ]:
#@markdown <center><h3>MEGA Downloader</h3></center>
#@markdown <br><center><img src='https://www.flaticon.com/svg/vstatic/svg/873/873184.svg?token=exp=1613311290~hmac=75e7382999e8d3c157c7db4a9d901c28' height="50" alt="Gdrive-logo"/></center>
import sys, os, urllib.request
import time
import subprocess
import contextlib
from IPython.display import clear_output

HOME = os.path.expanduser("~")
if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    runSh,
    loadingAn,
)
#@title MEGA public link download
URL = "" #@param {type:"string"}
OUTPUT_PATH = "" #@param {type:"string"}
if not OUTPUT_PATH:
  os.makedirs("downloads", exist_ok=True)
  OUTPUT_PATH = "downloads"
# MEGAcmd installing
if not os.path.exists("/usr/bin/mega-cmd"):
    loadingAn()
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
    clear_output()

# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']

def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out


def transfare():
    import codecs
    decoder = codecs.getincrementaldecoder("UTF-8")()
    cmd = ["mega-get", URL, OUTPUT_PATH]
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        print(line)
        


transfare()

# **Terminal And SSH**

In [ ]:
#@markdown <br><center><img src='https://raw.githubusercontent.com/butlerx/wetty/main/src/assets/favicon.ico' height="200" alt="wetty"/></center>
#@markdown <center><h3>Wetty<br />Terminal in browser over http/https.</h3></center><br>
import os, tarfile
import urllib.request
from IPython.display import clear_output
from subprocess import Popen

#####################################
USE_FREE_TOKEN = True  # @param {type:"boolean"}

TOKEN = ""  # @param {type:"string"}
REGION = "AP" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
PORT_FORWARD = "argotunnel" #@param ["ngrok", "localhost", "argotunnel"]
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")


from ocr import (
    loadingAn,
    PortForward_wrapper,
    findProcess,
)

loadingAn()
os.makedirs('tools/temp', exist_ok=True)

if not os.path.exists("tools/wetty/wetty"):
  ## Build then run
  # os.system("git clone https://github.com/butlerx/wetty.git tools/wetty")
  # Popen(
  #     'npm install'.split(), cwd='tools/wetty').wait()
  # Popen('npm run-script build'.split(), cwd='tools/wetty').wait()
  # Popen('npm i -g'.split(), cwd='tools/wetty').wait()

  ## run now
  wettyBF = 'https://github.com/biplobsd/temp/releases/download/v0.001/wetty.tar.gz'
  fileSN = 'tools/temp/wetty.tar.gz'
  urllib.request.urlretrieve(wettyBF, fileSN)
  with tarfile.open(fileSN, 'r:gz') as t:t.extractall('tools/')
  os.remove(fileSN)

if not findProcess("wetty", "--port"):
  # Popen(
  #     r'wetty --port 4343 --bypasshelmet \
  #     -b "/" -c "/bin/bash"'.split(), 
  #     cwd='/content')
  Popen(
      r'tools/wetty/wetty --port 4343 --bypasshelmet \
      -b "/" -c "/bin/bash"'.split(), 
      cwd='/content')

# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['wetty', '4343', 'http']], 
    REGION.lower(), 
    [f"{HOME}/.ngrok2/wetty.yml", 31199]
).start('wetty', displayB=True)

In [ ]:
#@markdown <h3>⬅️ Click Here to create ssh tunnel</h3>
#@markdown <br><center><img src='http://icons.iconarchive.com/icons/blackvariant/button-ui-system-apps/1024/Terminal-icon.png' height="200" alt="ssh"/></center>
#@markdown <center><h3>Forward ssh now</h3></center><br>
import os
import random
import string
import urllib.request
import time
from IPython.display import HTML, clear_output
from urllib.parse import urlparse

#####################################
USE_FREE_TOKEN = False
CREATE_VNC = True # @param {type:"boolean"}
CREATE_SSH = True # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
REGION = "IN" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
HOME = os.path.expanduser("~")
runW = get_ipython()

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    displayUrl,
    findProcess,
    CWD,
    textAn,
)

loadingAn()

# password ganarate
try:
  print(f"Found old password! : {password}")
except:
  password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))

if CREATE_SSH:
  USE_FREE_TOKEN = False

#Set root password
if CREATE_SSH  and os.path.exists('/var/run/sshd') == False:
  #Setup sshd
  runSh('apt install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen')
  runW.system_raw("echo root:$password | chpasswd")
  os.makedirs("/var/run/sshd", exist_ok=True)
  runW.system_raw('echo "PermitRootLogin yes" >> /etc/ssh/sshd_config')
  runW.system_raw('echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config')
  runW.system_raw('echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc')
  runW.system_raw('echo "export LD_LIBRARY_PATH" >> /root/.bashrc')

  #Run sshd
  if not findProcess("/usr/sbin/sshd", command="-D"):
    runSh('/usr/sbin/sshd -D &', shell=True)

if CREATE_VNC:
  clear_output()
  #start = time.time()
  textAn("Wait for almost 2 minutes. It's doing for VNC ready...")
  os.makedirs(f'{HOME}/.vnc', exist_ok=True)
  runW.system_raw('add-apt-repository -y ppa:apt-fast/stable < /dev/null')
  runW.system_raw('echo debconf apt-fast/maxdownloads string 16 | debconf-set-selections')
  runW.system_raw('echo debconf apt-fast/dlflag boolean true | debconf-set-selections')
  runW.system_raw('echo debconf apt-fast/aptmanager string apt-get | debconf-set-selections')
  runW.system_raw('apt install -y apt-fast')
  runW.system_raw('apt-fast install -y xfce4 xfce4-goodies firefox tightvncserver autocutsel')
  runW.system_raw(rf'echo "{password}" | vncpasswd -f > ~/.vnc/passwd')
  data = """
#!/bin/bash
xrdb $HOME/.Xresources
autocutsel -fork
startxfce4 &
"""
  with open(f'{HOME}/.vnc/xstartup', 'w+') as wNow:
    wNow.write(data)
  os.chmod(f'{HOME}/.vnc/xstartup', 0o755)
  os.chmod(f'{HOME}/.vnc/passwd', 0o400)
  runSh('sudo vncserver &', shell=True)
  runSh(f'git clone https://github.com/novnc/noVNC.git {CWD}/noVNC')
  runSh("bash noVNC/utils/launch.sh --listen 6080 --vnc localhost:5901 &",
        shell=True)
  #end = time.time()
# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
PORT_FORWARD = "argotunnel" #@param ["ngrok", "localhost", "argotunnel"]
Server = PortForward_wrapper(PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['ssh', 22, 'tcp'], 
               ['vnc', 6080, 'http']], REGION.lower(), 
               [f"{HOME}/.ngrok2/sshVnc.yml", 4455])
data = Server.start('ssh', displayB=False)

# output
data = urlparse(data['url'])
Host,port = data.hostname, str(data.port)
data2 = Server.start('vnc', displayB=False)
clear_output()
if CREATE_VNC:
  displayUrl(data2, pNamU='noVnc : ',
            EcUrl=f'/vnc.html?autoconnect=true&password={password}')
if CREATE_SSH:
  display(HTML("""<style>@import url('https://fonts.googleapis.com/css?family=Source+Code+Pro:200,900');  :root {   --text-color: hsla(210, 50%, 85%, 1);   --shadow-color: hsla(210, 40%, 52%, .4);   --btn-color: hsl(210, 80%, 42%);   --bg-color: #141218; }  * {   box-sizing: border-box; } button { position:relative; padding: 10px 20px;     border: none;   background: none;      font-family: "Source Code Pro";   font-weight: 900;font-size: 100%;     color: var(--text-color);      background-color: var(--btn-color);   box-shadow: var(--shadow-color) 2px 2px 22px;   border-radius: 4px;    z-index: 0;overflow: hidden; -webkit-user-select: text;-moz-user-select: text;-ms-user-select: text;user-select: text;}  button:focus {   outline-color: transparent;   box-shadow: var(--btn-color) 2px 2px 22px; }  .right::after, button::after {   content: var(--content);   display: block;   position: absolute;   white-space: nowrap;   padding: 40px 40px;   pointer-events:none; }  button::after{   font-weight: 200;   top: -30px;   left: -20px; }   .right, .left {   position: absolute;   width: 100%;   height: 100%;   top: 0; } .right {   left: 66%; } .left {   right: 66%; } .right::after {   top: -30px;   left: calc(-66% - 20px);      background-color: var(--bg-color);   color:transparent;   transition: transform .4s ease-out;   transform: translate(0, -90%) rotate(0deg) }  button:hover .right::after {   transform: translate(0, -47%) rotate(0deg) }  button .right:hover::after {   transform: translate(0, -50%) rotate(-7deg) }  button .left:hover ~ .right::after {   transform: translate(0, -50%) rotate(7deg) }  /* bubbles */ button::before {   content: '';   pointer-events: none;   opacity: .6;   background:     radial-gradient(circle at 20% 35%,  transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 75% 44%, transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 46% 52%, transparent 0, transparent 4px, var(--text-color) 5px, var(--text-color) 6px, transparent 6px);    width: 100%;   height: 300%;   top: 0;   left: 0;   position: absolute;   animation: bubbles 5s linear infinite both; }  @keyframes bubbles {   from {     transform: translate();   }   to {     transform: translate(0, -66.666%);   } }.zui-table {    border: solid 1px #DDEEEE;    border-collapse: collapse;    border-spacing: 0;    font: normal 13px;}.zui-table thead th {    background-color: #DDEFEF;    border: solid 1px #DDEEEE;    color: #0000009e;    padding: 10px;    text-align: left;}.zui-table tbody td {border: solid 1px #effff97a;color: #ffffffd1;    padding: 10px;}</style>"""))
  if 'argotunnel' is PORT_FORWARD:
    display(HTML("""<center><button> <table class="zui-table blueBG"> <p>Ssh config <p> <thead> <tr> <th>Host</th><th>Port</th> <th>Password</th></tr></thead> <tbody> <tr> <td>"""+Host+"""</td> <td>"""+port+"""</td> <td>"""+password+"""</td> </tr> </tbody> </table> <center> <br> <table class="zui-table blueBG"> <thead> <tr> <th></th> <th>Simple ssh command</th> </tr> </thead> <tbody> <tr> <td>Cloudflare tcp connect</td> <td>cloudflared access tcp --hostname """+Host+""" --url localhost:22</td> </tr> <tr> <td>Terminal connect</td> <td>ssh root@localhost -p 22</td> </tr> <tr> <td>Socks5 proxy</td> <td>ssh -D 8282 -q -C -N root@localhost -p 22</td> </tr> </tbody> </table> </center> <a target="_blank" style="text-decoration: none;color: hsla(210, 50%, 85%, 1);font-size: 10px;" href="https://raw.githubusercontent.com/biplobsd/OneClickRun/master/img/sshPreview_argotunnel.gif">NB. How to setup this's config. [Click ME]</a></button><center>"""))
  else:
    display(HTML("""<center><button> <table class="zui-table blueBG"> <p>Ssh config <p> <thead> <tr> <th>Host</th> <th>Port</th> <th>Password</th> </tr> </thead> <tbody> <tr> <td>"""+Host+"""</td> <td>"""+port+"""</td> <td>"""+password+"""</td> </tr> </tbody> </table> <center> <br> <table class="zui-table blueBG"> <thead> <tr> <th></th> <th>Simple ssh command</th> </tr> </thead> <tbody> <tr> <td>Terminal connect</td> <td>ssh root@"""+Host+""" -p """+port+"""</td> </tr> <tr> <td>Socks5 proxy</td> <td>ssh -D 8282 -q -C -N root@"""+Host+""" -p """+port+"""</td> </tr> </tbody> </table> </center> <a target="_blank" style="text-decoration: none;color: hsla(210, 50%, 85%, 1);font-size: 10px;" href="https://raw.githubusercontent.com/biplobsd/OneClickRun/master/img/sshPreview.gif">NB. How to setup this's config. [Click ME]</a></button><center>""")

#Local upload


In [ ]:
#@markdown <h3>Upload
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))